In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_reader import read_consumption_and_weather, plot_consumption, plot_consumption_and_weather
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [10]:
df = read_consumption_and_weather()[-1]
df.head()

Holes by length and occurrences in column NO3:
holes
1        1
dtype: int64
Filling holes up to length 3
Any remaining holes after interpolation? False


mba                                NO1                     NO2              \
                           consumption temperature consumption temperature   
2017-05-01 00:00:00+00:00  3325.431995         2.2    3534.588    4.500000   
2017-05-01 01:00:00+00:00  3344.690998         3.2    3560.209    5.500000   
2017-05-01 02:00:00+00:00  3398.359002         4.0    3596.149    3.200000   
2017-05-01 03:00:00+00:00  3430.220001         3.6    3594.042    5.100000   
2017-05-01 04:00:00+00:00  3606.750000         3.4    3665.091    3.866667   

mba                               NO3                     NO4              \
                          consumption temperature consumption temperature   
2017-05-01 00:00:00+00:00    2674.838    5.250000    2061.209    0.150000   
2017-05-01 01:00:00+00:00    2678.188    4.000000    2045.436    0.050000   
2017-05-01 02:00:00+00:00    2678.341    3.350000    2020.007    0.150000   
2017-05-01 03:00:00+00:00    2684.172    3.066667    2021.920    0.100000   
2017-05-01 04:00:00+00:00    2733.345    2.933333    2045.913    0.866667   

mba                               NO5              
                          consumption temperature  
2017-05-01 00:00:00+00:00    1609.089         3.9  
2017-05-01 01:00:00+00:00    1623.485         3.2  
2017-05-01 02:00:00+00:00    1634.128         2.7  
2017-05-01 03:00:00+00:00    1647.906         2.4  
2017-05-01 04:00:00+00:00    1679.176         2.3

In [11]:
df_no1 = df['NO1']
df_no1.loc[:,'region'] = 1

df_no2 = df['NO2']
df_no2.loc[:,'region'] = 2

df_no3 = df['NO3']
df_no3.loc[:,'region'] = 3

df_no4 = df['NO4']
df_no4.loc[:,'region'] = 4

df_no5 = df['NO5']
df_no5.loc[:,'region'] = 5

# combine all dataframes into one
df = pd.concat([df_no1, df_no2, df_no3, df_no4, df_no5])
df.head()

c:\Python39\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,consumption,temperature,region
2017-05-01 00:00:00+00:00,3325.431995,2.2,1
2017-05-01 01:00:00+00:00,3344.690998,3.2,1
2017-05-01 02:00:00+00:00,3398.359002,4.0,1
2017-05-01 03:00:00+00:00,3430.220001,3.6,1
2017-05-01 04:00:00+00:00,3606.750000,3.4,1


In [12]:
# datetime features
dates = df.index
df['time_of_day'] = df.index.hour
df['time_of_week'] = df.index.dayofweek
df['time_of_month'] = df.index.day
df['time_of_year'] = df.index.month

df['consumption_24h_lag'] = df['consumption'].shift(24).fillna(method='bfill')
df['consumption_yday_mean'] = df['consumption'].shift(24).rolling(24).mean().fillna(method='bfill')
df['consumption_prev'] = df['consumption'].shift(1).fillna(method='bfill')
df.head()


,consumption,temperature,region,time_of_day,time_of_week,time_of_month,time_of_year,consumption_24h_lag,consumption_yday_mean,consumption_prev
2017-05-01 00:00:00+00:00,3325.431995,2.2,1,0,0,1,5,3325.431995,3565.553874,3325.431995
2017-05-01 01:00:00+00:00,3344.690998,3.2,1,1,0,1,5,3325.431995,3565.553874,3325.431995
2017-05-01 02:00:00+00:00,3398.359002,4.0,1,2,0,1,5,3325.431995,3565.553874,3344.690998
2017-05-01 03:00:00+00:00,3430.220001,3.6,1,3,0,1,5,3325.431995,3565.553874,3398.359002
2017-05-01 04:00:00+00:00,3606.750000,3.4,1,4,0,1,5,3325.431995,3565.553874,3430.220001


In [13]:
# Fit scalers
scalers = {}
columns_to_scale = ['consumption', 'temperature', 'consumption_24h_lag', 'consumption_yday_mean', 'consumption_prev']
for x in df[columns_to_scale]:
  scalers[x] = StandardScaler().fit(df[x].values.reshape(-1, 1))

# Transform data via scalers
norm_df = df.copy()
for key in scalers.keys():
  scaler = scalers[key]
  norm = scaler.transform(norm_df[key].values.reshape(-1, 1))
  norm_df[key] = norm.flatten()

norm_df.head()

# inverse
# def inverse_transform(data, scaler, col):
#     norm = scaler.transform(data[[col]])
#     return scaler.inverse_transform(norm).flatten()

# for col in columns_to_scale:
#   norm_data[col] = inverse_transform(norm_data, scalers[col], col)

,consumption,temperature,region,time_of_day,time_of_week,time_of_month,time_of_year,consumption_24h_lag,consumption_yday_mean,consumption_prev
2017-05-01 00:00:00+00:00,0.219890,-0.700416,1,0,0,1,5,0.219809,0.433275,0.219886
2017-05-01 01:00:00+00:00,0.236287,-0.563661,1,1,0,1,5,0.219809,0.433275,0.219886
2017-05-01 02:00:00+00:00,0.281981,-0.454256,1,2,0,1,5,0.219809,0.433275,0.236283
2017-05-01 03:00:00+00:00,0.309108,-0.508958,1,3,0,1,5,0.219809,0.433275,0.281977
2017-05-01 04:00:00+00:00,0.459408,-0.536309,1,4,0,1,5,0.219809,0.433275,0.309104


In [14]:
norm_df.to_csv('data/norm_df.csv', index_label=False)

In [15]:
simple_df = norm_df[['consumption']]
simple_df.to_csv('data/simple_df.csv')

In [16]:
norm_df.index

DatetimeIndex(['2017-05-01 00:00:00+00:00', '2017-05-01 01:00:00+00:00',
               '2017-05-01 02:00:00+00:00', '2017-05-01 03:00:00+00:00',
               '2017-05-01 04:00:00+00:00', '2017-05-01 05:00:00+00:00',
               '2017-05-01 06:00:00+00:00', '2017-05-01 07:00:00+00:00',
               '2017-05-01 08:00:00+00:00', '2017-05-01 09:00:00+00:00',
               ...
               '2023-01-21 14:00:00+00:00', '2023-01-21 15:00:00+00:00',
               '2023-01-21 16:00:00+00:00', '2023-01-21 17:00:00+00:00',
               '2023-01-21 18:00:00+00:00', '2023-01-21 19:00:00+00:00',
               '2023-01-21 20:00:00+00:00', '2023-01-21 21:00:00+00:00',
               '2023-01-21 22:00:00+00:00', '2023-01-21 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=251040, freq=None)